In [28]:
from all_comps import climbing_category, competition_overview_df, pd, np, requests

# Creating a stage_results dataframe
stage_results_df = pd.DataFrame()

# Complete competition_overview_df with added information.
for comp in competition_overview_df.iterrows():
    base_url = 'http://egw.ifsc-climbing.org/egw/ranking/json.php?'
    competition_id = comp[1]['compId']
#     competition_id = '7923'

    if comp[1]['isBoulderingComp'] == 1:
        # Men and women bouldering
        for category in ['5', '6']:
            df_category = climbing_category(category)

            # Both qualies, semi-finals and finals 
            for stage in ['0', '1', '2', '3']:
                df_stage = competition_stage(stage)

                url = base_url + 'comp=' + competition_id + '&cat=' + category + '&route=' + stage
                response = requests.get(url)
                
                # Checking if the url returns a value (for the comps that don't have the same format)
                if response.status_code != 200:
                    continue
                comp_json = response.json()
                
                # Adding stage information to main competition overview dataframe
                competition_overview_df.loc[competition_overview_df['compId'] == competition_id, 'isoOpen' + df_stage + df_category] = comp_json.get('route_iso_open')
                competition_overview_df.loc[competition_overview_df['compId'] == competition_id, 'isoClose' + df_stage + df_category] = comp_json.get('route_iso_close')
                competition_overview_df.loc[competition_overview_df['compId'] == competition_id, 'stageStart' + df_stage + df_category] = comp_json.get('route_start')
                competition_overview_df.loc[competition_overview_df['compId'] == competition_id, 'numProblems' + df_stage + df_category] = comp_json.get('route_num_problems')
                competition_overview_df.loc[competition_overview_df['compId'] == competition_id, 'climbingTime' + df_stage + df_category] = comp_json.get('route_climbing_time')
                
                stage_results_df.append(get_stage_results(comp_json), ignore_index=True)
# competition_overview_df
# comp_json


In [36]:
url = 'http://egw.ifsc-climbing.org/egw/ranking/json.php?comp=7923&cat=6&route=3'
stage_results_df.append(get_stage_results(requests.get(url).json()))

,boulder1,boulder2,boulder3,boulder4,boulder5,category,climberID,compId,rankAfterStage,stage,startOrder
0,t5 b1,b0,t1 b1,t3 b3,None,Men,5781,983,1,F,3
1,t1 b1,b0,t3 b1,b0,None,Men,2602,983,2,F,1
2,b1,b0,t1 b1,b0,None,Men,2787,983,3,F,6
3,b1,b0,t3 b1,b0,None,Men,6096,983,4,F,5
4,b6,b0,t4 b1,b0,None,Men,6933,983,5,F,4
5,b1,b0,t5 b2,b0,None,Men,6489,983,6,F,2


In [6]:
def get_stage_results(url_json):
    new_stage_rows = []    
    for climber in comp_json['participants']:
        new_comp_stage = dict()
        
        # This same information needs to be in the row for every climber
        new_comp_stage.update({
            'compId': comp_json.get('WetId'),
            'category': climbing_category(comp_json.get('GrpId')),
            'stage': competition_stage(comp_json['route_order'])
        })

        # Get climber's results for this stage
        new_comp_stage.update({
            'climberID': climber.get('PerId'),
            'startOrder': climber.get('start_order'),
            'rankAfterStage' : climber.get('result_rank'),
            'boulder1': climber.get('boulder1'),
            'boulder2': climber.get('boulder2'),
            'boulder3': climber.get('boulder3'),
            'boulder4': climber.get('boulder4'),
            'boulder5': climber.get('boulder5'),
        })
        
        # Append new row dictionary, (representing a stage of a comp), in the new_stage_rows list
        new_stage_rows.append(new_comp_stage)

    # Return new rows to insert in dataframe
    return new_stage_rows

In [5]:
def competition_stage(stage, intext=1):
    external_stages = {
        '-1': 'General result',
        '0': '1. Qualification',
        '1': '2. Qualification',
        '2': 'Semi-Final',
        '3': 'Final'
    }
    df_stages = {
        '0': 'Q1',
        '1': 'Q2',
        '2': 'SF',
        '3': 'F'
    }
    return df_stages.get(stage, 'Other') if intext == 1 else external_stages.get(stage, 'Other')